# Motivation for Ansible
## Automating Device Management
- It all started with servers in the server room and administrator managing it
- The server room become a datacenter
- And after cloud came Internet of Things
- How are you going to manage that is a consistent way?

## Ansible vs. Shell Scripts
- Modern day datacenters need automation
- Shell scripting is feasible in small environments only
- - Can you install 3 servers as a webserver with a database using shell scripts?
- - How easy is it to put different parameters in shell scripts?
- - What can you do with a shellscript if and administrator makes an error and removes configuration?
- - How easy is it to use a shell script deploying a desired state?
- - What can you od with shell scripts across different Linux Distributions?
- The answer lies in Configuration Management
- In Configuration Management, you'll define a desired state. If current state changes, you'll just re-apply the desired state.

## Ansible vs. Puppet and Others
- Ansible is just one of the configuration managemnet solutions
- Other solutions are Puppet, Chef, Salt, CFEngine and more
- Is Ansible really better than the others?
- It is definitely easier than many others
- And it doesn't need an agent on manged servers, but uses SSH
- Ansible is modular which makes it fexible
- And the modules by defult written in Python
- Ober 1000 modules are already available and administrators who know Pyothon can develop their own Ansible modules
 
## Beyond just Linux
- Using Ansible allows you to manage anything
- The origins were in managing Linux through SSH
- Using different plugins, many other assets can be managed
- Different assets can be managed by direct API access
- In the end that helps managing the enterprise through Ansible
- Modules are written for very differetn assets, to speak the language of the specific managed devices

# Create Virtual-Machines
- Install VirtualBox, and set up 3 Virtual Machines
- - Controller Virtual Machine
- - Node1 Virtual Machine
- - Node2 Virtual Machine
- Follow the instruction to create 3 CEntOS 7.3 Virtual Machine from the "Virtual Machine" Tutorial in this repository

# Set up SSH communication between Virtual Machines
- First add the Hostnames and IP address of our virtual machines, to the local DNS of our virtual machine by editing the **/etc/hosts** file

### Configuring SSH
- Set up SSH Key-based authentication 
- - **ssh-keygen**
- This creates a public key as well as a private key
- - The server that has the public key sends a challenge that can only be answered with the private key
- - Keey the private key in the local user account on the control node
- - Send the publkic key to the **~/.ssh/authorized_keys** file in the target user home directory
- - - Use **ssh-copy-id user@remotehost**
- - - Notice that the lodcal user name and the remote user name do NOT have to be the same
- Don't forget to include the controller host as well if you want to manage that also

Add the user to wheel, in every VM run

In [ ]:
%%bash
su -
visudo # check if wheel in enabled
usermod -aG wheel controller # run in controller vm
usermod -aG wheel node1 # run in node1 vm
usermod -aG wheel node2 # run in node2 vm

Restart all the VMs

Now make sure that all the VMs are up
- SSH into the controller VM

In [ ]:
%%bash
su -
vi /etc/hosts

Enter the configurations in the hosts file

 Copy the hosts file to other vms

In [ ]:
%%bash
scp /etc/hosts node1:/etc/
scp /etc/hosts node2:/etc/

Setup SSH

In [ ]:
%%bash
su controller
ssh-keygen # just use default settings
ssh-copy-id node1@node1.example.com
ssh-copy-id node2@node2.example.com

### Now we can ssh into node1 and node2 from controller without being prompt for password or passphrase

In [ ]:
%%bash
ssh node1@node1
ssh node2@node2